In [4]:
import os
import ebooklib
from ebooklib import epub
import tkinter as tk
from tkinter import filedialog
import shutil
import re
from lxml import html
import posixpath


In [ ]:
def input_epub():
    root = tk.Tk()
    root.withdraw()
    root.wm_attributes("-topmost", True)
    root.update_idletasks()

    file_path = filedialog.askopenfilename(
        parent=root,
        title="Seleziona il file EPUB",
        filetypes=[("EPUB", "*.epub"), ("Tutti i file", "*.*")]
    )

    root.destroy()

    if not file_path:
        return None

    return epub.read_epub(file_path)


def export_epub_folder(book, out_dir="epub_export", export=False):

    if not export:
        return out_dir
    
    os.makedirs(out_dir, exist_ok=True)

    for item in book.get_items():
        name = item.get_name()  # es. "text/part0001.xhtml", "images/a.jpg"
        path = os.path.join(out_dir, *name.split("/"))
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "wb") as f:
            f.write(item.get_content())

    return out_dir


def is_relative(url: str) -> bool:
    if not url:
        return False
    u = url.strip().lower()
    return not (u.startswith(("http://", "https://", "mailto:", "tel:", "data:", "#", "/")))


def make_full_html(book, out_dir="epub_export", out_name="full.html"):
     # --- 1) Ricaviamo l'ordine di lettura dall'EPUB ---
    # book.spine è una lista di tuple (idref, linear_flag)
    # idref punta a un item nel manifest (di solito un documento HTML/XHTML).
    # "nav" e "ncx" sono file di navigazione (indice), non capitoli di lettura.
    spine_ids = [idref for (idref, _) in book.spine if idref not in ("nav", "ncx")]

    # piccola curiosità, gli idred non iniziano da id1, ma da un idX, poichè gli id iniziali 
    # sono le immagini che vengono solo richiamate tramite dal file della pagina .html cosi: src="../images/00001.jpeg"
    # quindi idrefX di un itemref di uno spine sono link a delle pagine .html che contengono testo, immagini, css, stile, ecc..
    # e che quindi fornisce l'ordine di lettura
    # quindi quello che dovremmo fare è solo un iterazione di tutto lo spine, e di tutti 

    css_links = [] #lista vuota di link dentro gli item per il css
    inline_styles = [] #lista vuota di link dentro gli item per il lo stile inline
    body_parts = [] #lista vuota di link dentro gli item per il lo stile inline

    # Cicliamo tutti i "capitoli" nello spine ---
    for idref in spine_ids:
        # Recupera l'oggetto item dal libro usando l'id dello spine
        item = book.get_item_with_id(idref)

        # Se l'item non esiste o non è un documento (html/xhtml), lo saltiamo
        # ebooklib.ITEM_DOCUMENT indica un contenuto tipo pagina/chapters
        if not item or item.get_type() != ebooklib.ITEM_DOCUMENT:
            continue

        # Percorso interno dell'item dentro l'epub (es: "Text/chapter1.xhtml")
        chapter_path = item.get_name()

        # Directory del capitolo (serve per risolvere i percorsi relativi)
        # es: "Text" se chapter_path è "Text/chapter1.xhtml"
        base_dir = posixpath.dirname(chapter_path)

        # Parse del contenuto HTML/XHTML in un DOM (albero) manipolabile con XPath
        doc = html.fromstring(item.get_content())

        # Raccogliamo i CSS esterni (<link rel="stylesheet" href="...">) 
        for link in doc.xpath('//link[@rel="stylesheet"]'):
            href = link.get("href")

            # Se l'href è relativo (es: "../Styles/style.css"), lo convertiamo
            # in un percorso normalizzato basato sulla directory del capitolo
            if href and is_relative(href):
                href = posixpath.normpath(posixpath.join(base_dir, href))

            # Salviamo il link (anche se assoluto, se presente)
            if href:
                css_links.append(href)

        # Raccogliamo CSS inline (<style> ... </style>) 
        for style in doc.xpath("//style"):
            # style.text contiene il testo dentro il tag <style>
            css = (style.text or "").strip()
            if css:
                inline_styles.append(css)

        # Sistemiamo i percorsi relativi delle risorse nel body 
        # per esempio: <img src="../images/00001.jpeg"> deve diventare "Text/../images/00001.jpeg" normalizzato.
        # Questo è importante perché dopo aver unito tutto in un solo HTML, i path devono puntare
        # a risorse coerenti rispetto alla struttura esportata.
        
        for el in doc.xpath('//*[@src]'):
            src = el.get("src")
            if src and is_relative(src):
                el.set("src", posixpath.normpath(posixpath.join(base_dir, src)))

        # Stesso discorso per i link (<a href="...">) e altri elementi con href
        for el in doc.xpath('//*[@href]'):
            href = el.get("href")
            if href and is_relative(href):
                el.set("href", posixpath.normpath(posixpath.join(base_dir, href)))

        # Estrarre il contenuto del <body> del capitolo
        body = doc.find(".//body")
        if body is not None:
            # Con doc.find(".//body") otteniamo l'elemento <body>
            # Ora vogliamo concatenare SOLO i figli del body, non l'intero tag <body> stesso.
            # html.tostring(child) serializza ogni nodo figlio in HTML (stringa).
            inner = "\n".join(
                html.tostring(child, encoding="unicode")
                for child in body
            )
            body_parts.append(inner)

    # Rimozione duplicati
    # Molti capitoli linkano lo stesso CSS: evitiamo di includerlo più volte nell'head.
    seen = set()
    css_links_unique = []
    for c in css_links:
        if c not in seen:
            seen.add(c)
            css_links_unique.append(c)

    # Creiamo le righe <link> da mettere nell'HEAD dell'HTML finale
    head_css = "\n".join(f'<link rel="stylesheet" href="{c}">' for c in css_links_unique)

    # Uniamo tutti i CSS inline in un unico <style>
    head_inline = ""
    if inline_styles:
        # Separiamo i blocchi di stile con una riga vuota per leggibilità
        head_inline = "<style>\n" + "\n\n".join(inline_styles) + "\n</style>"

    #  Costruiamo l'HTML finale
    # body_parts viene unito con <hr> per separare visivamente i capitoli
    full = f"""<!doctype html>
<html>
<head>
<meta charset="utf-8">
<title>EPUB(unito)</title>
{head_css}
{head_inline}
</head>
<body>
{"<hr>".join(body_parts)}
</body>
</html>"""

    #  Scriviamo su file
    out_path = os.path.join(out_dir, out_name)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(full)

    # Ritorniamo il path del file generato
    return out_path

In [ ]:

book = input_epub()
if book is None:
    print("Nessun file selezionato.")
else:
    out_dir = export_epub_folder(book, "epub_export", export=True)
    full_path = make_full_html(book, out_dir, "full.html")
    print("Creato:", os.path.abspath(full_path))

    # apre nel browser (Windows)
    #os.startfile(os.path.abspath(full_path))

Creato: c:\Users\utente\Desktop\epub\epub_export\full.html
